### Importing libraries

In [1]:
import matplotlib.pyplot as pp
import pycbc.noise
import pycbc.psd
import pycbc.filter
from pycbc.waveform import get_td_waveform
from pycbc.waveform import get_fd_waveform
import numpy as np
from pycbc.vetoes import power_chisq
from pycbc.events.ranking import newsnr
import pandas as pd
from pycbc.filter import sigma


PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


# Generating noise

In [2]:
def gen_noise(psd_need):
    
    # The color of the noise matches a PSD which you provide
    flow = 30.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)

    if (psd_need == True) :
        pp.plot(psd.sample_frequencies,psd.data)
        pp.loglog()
        pp.xlim(30,2000)
        pp.ylabel('$Strain^2 / Hz$')
        pp.xlabel('Frequency (Hz)')
        pp.show()


    # Generate 512 seconds of noise at 4096 Hz
    delta_t = 1.0 / 4096
    tsamples = int(512/ delta_t)
    ts = pycbc.noise.gaussian.noise_from_psd(tsamples,delta_t, psd, seed=127)
    
    
    #plotting noise 
#     pp.plot(ts.sample_times, ts)
#     pp.ylabel('Strain')
#     pp.xlabel('Time (simport numpy as np

#     pp.show()
    

    return ts



# Generating strain

In [3]:
def gen_waveform(model,m1,m2,s1z,s2z,distance,time,l):
    
    from pycbc.detector import Detector
    from pycbc.waveform import get_td_waveform


   
    ra = 1.7
    dec = 1.7
    pol = 0.2
    inclination_1 = 0




    # We can calcualate the antenna pattern for Hanford at
    # the specific sky location
    d = Detector("H1")

    # We get back the fp and fc antenna pattern weights.
    fp, fc = d.antenna_pattern(ra, dec, pol, time)
    #print("fp={}, fc={}".format(fp, fc))

    # These factors allow us to project a signal into what the detector would observe
    # IMRPhenomPv2
    # Generate the waveform for the BNS merger
    hp_1, hc_1 = get_td_waveform(approximant=model,
                             mass1=m1, mass2=m2,spin1z=s1z,spin2z=s2z,
                             distance=distance, inclination=inclination_1,
                             delta_t=1.0/4096, f_lower=30)

    ## Apply the factors to get the detector frame strain
    ht_1 = fp * hp_1 + fc * hc_1
    
    
#     pp.plot(ht_1.sample_times, ht_1, label=l)
#     pp.legend()
#     pp.ylabel('Strain')
#     pp.xlabel('Time')
#     pp.grid()
#     pp.show()
    
    
    
    return ht_1


# Creating overlapping signal

In [4]:
def overlap(ht_1,ht_2,buffer):
    
    #Equal lengths of signals
    ht_2.resize(len(ht_1))
    
    #Shifting the merger time
    ht_2=ht_2.cyclic_time_shift(ht_2.end_time - buffer)
    
    #Equating the start time of both signals
    ht_2.start_time = ht_1.start_time
    
    #Combining the signals
    ht_total = ht_1 + ht_2

    #Ploting the signals
#     pp.plot(ht_total.sample_times, ht_total, label='Overlapped')
#     pp.legend()
#     pp.title('Overlapped Signal')
#     pp.ylabel('Strain')
#     pp.xlabel('Time')
#     pp.grid()
#     pp.show()
    
    return ht_total,ht_2


# Injecting strain into noise

In [5]:
def inject(ht_total,ts):
    
    #Resizing the signal
    ht_total.resize(len(ts)) 
    
    #Equating the start time
    ht_total.start_time = ts.start_time
    
    #Injecting signal into noise
    ts = ts.add_into(ht_total) 
    
    #plotting the result
#     pp.plot(ts.sample_times, ts)
#     pp.xlabel('Time(s)')
#     pp.ylabel('Strain')
#     pp.grid()
#     pp.show()
    
    return ts 




# Calculating PSD

In [ ]:
# def psd_data(conditioned):

    from pycbc.psd import interpolate, inverse_spectrum_truncation
    # Estimate the power spectral density

    # We use 4 second samples of our time series in Welch method.
    psd = conditioned.psd(4)

    # Now that we have the psd we need to interpolate it to match our data
    # and then limit the filter length of 1 / PSD. After this, we can
    # directly use this PSD to filter the data in a controlled manner

    psd = interpolate(psd, conditioned.delta_f)

    # 1/PSD will now act as a filter with an effective length of 4 seconds
    # Since the data has been highpassed above 30 Hz, and will have low values
    # below this we need to informat the function to not include frequencies
    # below this frequency. 
    psd = inverse_spectrum_truncation(psd, int(4 * conditioned.sample_rate),
                                      low_frequency_cutoff=30)

#     pp.loglog(psd.sample_frequencies, psd)
#     pp.ylabel('$Strain^2 / Hz$')
#     pp.xlabel('Frequency (Hz)')
#     pp.xlim(30, 1024)
    
    return psd

# Creating template bank

In [7]:

def template_bank(m_1,m_2,m_radius,temp_numb):

    # Generate templates for BNS
    temp_bank = np.zeros((temp_numb, 2))
    for i in range(temp_numb):
        r1 = np.random.uniform(0,m_radius)
        r2 = np.random.uniform(0,(np.pi*2))           
        m1 = r1*(np.cos(r2)) + m_1
        m2 = r1*(np.sin(r2)) + m_2

        temp_bank[i, :] = [m1, m2]

    temp_bank[i,:] = [m_1,m_2]
    
    return temp_bank



# Generating template

In [8]:
def gen_template(model,m1,m2,s1z,s2z,conditioned):
    
    from pycbc.detector import Detector
    from pycbc.waveform import get_td_waveform

    
    ra = 1.7
    dec = 1.7
    pol = 0.2
    inclination_1 = 0
    time=0.0



    # We can calcualate the antenna pattern for Hanford at
    # the specific sky location
    d = Detector("H1")

    # We get back the fp and fc antenna pattern weights.
    fp, fc = d.antenna_pattern(ra, dec, pol, time)
    
    
    
    
    hp, hc = get_td_waveform(approximant=model,
                     mass1=m1,
                     mass2=m2,spin1z=s1z,spin2z=s2z,
                     delta_t=conditioned.delta_t,
                     f_lower=30)
    
    
    ht_template = fp * hp + fc * hc
    
    #Resizing the template to match data
    ht_template.resize(len(conditioned))
    
    #Time shift 
    template = ht_template.cyclic_time_shift(ht_template.start_time)
    
   
    return template


# Generating SNR

In [9]:
def gen_SNR(template,conditioned,psd):
    from pycbc.filter import matched_filter
    import numpy

    snr = matched_filter(template, conditioned,
                         psd=psd, low_frequency_cutoff=30)



    # The `matched_filter` function actually returns a 'complex' SNR.
    # What that means is that the real portion correponds to the SNR
    # associated with directly filtering the template with the data.
    # The imaginary portion corresponds to filtering with a template that
    # is 90 degrees out of phase. Since the phase of a signal may be 
    # anything, we choose to maximize over the phase of the signal.

   
    peak = abs(snr).numpy().argmax()
    snrp = abs(snr[peak])
    time = snr.sample_times[peak]
   
    
    return snr,time,peak,snrp

# Chi-square Test

In [10]:
def gen_chisquare(temp,data,m_1,m_2,s_1,s_2,psd):
    
    n =int(0.72*pycbc.pnutils.get_freq('fSEOBNRv4Peak',m_1,m_2,s_1,s_2)**0.7)
    
    chisq = power_chisq(temp, data, n, psd=psd, low_frequency_cutoff=30.0)

    # convert to a reduced chisq
    chisq /= (n * 2) - 2
    
    
    
    return chisq


# Calculating SNR,Chi-square,New SNR for entire template bank

In [11]:
def snr_chisq_nsnr(num_templates,temp_bank,psd,data,model,s):

    template_info = []


    for i in range(num_templates):
        T1 = gen_template(model,temp_bank[i][0],temp_bank[i][1],s,s,data)

        S1,time_bns,index_peak_bns,old_snr_bns = gen_SNR(T1,data,psd)

        chisq_1 = gen_chisquare(T1,data,temp_bank[i,:][0],temp_bank[i,:][1],s,s,psd)
        chisq_val_bns = chisq_1[index_peak_bns]

        new_snr_bns = newsnr(abs(S1),chisq_1)

        snrp_bns = new_snr_bns[index_peak_bns]

        template_info.append([temp_bank[i,:][0],temp_bank[i,:][1],time_bns,old_snr_bns,snrp_bns,chisq_val_bns])

    return template_info
    

# Recovery of signal

In [12]:
def recovery(data,n,bns_bank,bbh_bank,psd): 
    
  

    bns_info =  snr_chisq_nsnr(n,bns_bank,psd,data,'TaylorF2',0.05)

    bbh_info =  snr_chisq_nsnr(n,bbh_bank,psd,data,'IMRPhenomPv2',0.8)

    total_info = [*bbh_info,*bns_info]
    
    

    info = {}
    for i in total_info:
        info.update({i[4] : [i[0],i[1],i[2],i[3],i[5]]})


    snrp_1 = max(info.keys())
    time_1 = info.get(max(info.keys()))[2]
    m1_1 = info.get(max(info.keys()))[0]
    m2_1 = info.get(max(info.keys()))[1]
    chi_sq_1 = info.get(max(info.keys()))[4]

    print("The signal detected is due to the following masses ",m1_1,"M and ", m2_1, "M at time ", time_1, "seconds", " with new SNR value as", snrp_1, " and chi~square value as ", chi_sq_1 )
    
    
    return m1_1,m2_1,time_1,snrp_1,total_info



# Aligning and subtracting the recovered signal to find the other signal

In [13]:
def align_sub(m1_1,m2_1,time_1,snrp_1,data,psd):   
    
    # The time, amplitude, and phase of the SNR peak tell us how to align
    # our proposed signal with the data.

    # Shift the template to the peak time
    dt = time_1 - data.start_time 
    
    
    if (m1_1 >= 15 and m2_1 >= 15) : 
        temp_sub  = gen_template('IMRPhenomPv2',m1_1,m2_1,0.8,0.8,data)
        
    else :
        temp_sub = gen_template('TaylorF2',m1_1,m2_1,0.05,0.05,data)
        
     
    
    
    aligned = temp_sub.cyclic_time_shift(dt)

    # scale the template so that it would have SNR 1 in this data
    aligned /= sigma(aligned, psd=psd, low_frequency_cutoff=30.0)

    # Scale the template amplitude and phase to the peak value
    aligned = (aligned.to_frequencyseries() * snrp_1).to_timeseries()
    aligned.start_time = data.start_time

    
    subtracted = data - aligned
    
    psd2 = psd_data(subtracted)  

    
    return subtracted,psd2

# Creating Data

### Noise generation followed by creating 2 signals i.e. BNS and BBH; overlapping them in such a way that their merger peaks differ by 5 seconds,followed by injection of this overlapped signal into noise.Creating a template bank and calculating psd for this data(strain+noise)


In [14]:
noise = gen_noise(False)

h1_bns = gen_waveform('TaylorF2',1.4,1.3,0.05,0.05,350,0.0,"BNS")

h1_bbh = gen_waveform('IMRPhenomPv2',30,40,0.8,0.8,5500,0.0,"BBH")

ht_total,h1_bbh = overlap(h1_bns,h1_bbh,5)

ts1 = inject(ht_total,noise)

bns_templates = template_bank(1.4,1.3,0.25,3)

bbh_templates = template_bank(30,40,10,3)

psd1 = psd_data(ts1)  

# Initiating the recovery process

In [15]:

mass_1,mass_2,time,snrp,total = recovery(ts1,3,bns_templates,bbh_templates,psd1)



The signal detected is due to the following masses  30.0 M and  40.0 M at time  251.000244140625 seconds  with new SNR value as 8.635529529314473  and chi~square value as  0.9272549625664297


In [16]:
for i in range(3):
   
    sub,psd_sub = align_sub(mass_1,mass_2,time,snrp,ts1,psd1)
    M1,M2,T,SP,Tot = recovery(sub,3,bns_templates,bbh_templates,psd1)
    mass_1=M1
    mass_2=M2
    time=T
    snrp=SP
    
    total.extend([[0,0,0,0,0,0]])
    total.extend(Tot)
    
    print(" ")

The signal detected is due to the following masses  1.4 M and  1.3 M at time  255.800048828125 seconds  with new SNR value as 7.445981973111216  and chi~square value as  1.1122420684957792
 
The signal detected is due to the following masses  30.0 M and  40.0 M at time  251.000244140625 seconds  with new SNR value as 8.583843222228893  and chi~square value as  0.915250618212153
 
The signal detected is due to the following masses  1.4 M and  1.3 M at time  255.800048828125 seconds  with new SNR value as 7.446576219573375  and chi~square value as  1.1122198695132448
 


In [17]:
for i in total:
    print(i)

[30.353882553083054, 39.06568383941927, 251.00048828125, 8.586350397185385, 8.565235585626157, 1.0098248308464426]
[23.262139118038547, 44.573157432755984, 251.001220703125, 6.608186202462286, 5.969774393955021, 1.3889249414187717]
[30.0, 40.0, 251.000244140625, 8.635529529314473, 8.635529529314473, 0.9272549625664297]
[1.336897421661336, 1.1820520959047067, 361.46240234375, 5.374611333377173, 5.374611333377173, 0.9805070851506373]
[1.3858629021499906, 1.433842431951303, 192.181640625, 5.378132271666368, 5.246802963940404, 1.0969056031012396]
[1.4, 1.3, 255.800048828125, 7.758350086509608, 7.526116896691577, 1.1187025324362552]
[0, 0, 0, 0, 0, 0]
[30.353882553083054, 39.06568383941927, 240.380615234375, 5.094484187370699, 5.013291210098589, 1.0633628199406988]
[23.262139118038547, 44.573157432755984, 504.40625, 5.141744027258681, 5.141744027258681, 0.853623482314273]
[30.0, 40.0, 240.38037109375, 5.122937006169674, 5.059029370958772, 1.0496468677480255]
[1.336897421661336, 1.1820520959

### Storing the values of every iteration to an excel file

In [18]:
df_total_info = pd.DataFrame(total,columns = ['M1','M2','Time','Old_SNR','New_SNR','Chi-Sq.'])

df_total_info.to_excel('Iterative_Signal_Recovery.xlsx',index = False)